In [8]:
import math
import numpy as np

In [9]:
EPS = 1e-7
ITERATIONS = 200000

In [10]:
def _format_matrix(min, max, size):
    return np.array((max - min) * np.random.random((size, size)) + min, dtype=np.float64)
    
def rand_matrix(size):
    A = np.array(np.random.rand(size, size), dtype=np.float64)
    A = np.tril(A) + np.tril(A, -1).T
    return A.T * A

def diag_matrix(size):
    A = _format_matrix(0.0, 10.0, size)
    for i in range(size):
        A[i, i] = sum(A[i]) + 1
    return A

def hilbert_matrix(size):
    ans = np.array(np.zeros((size, size)), dtype=np.float64)
    for i in range(size):
        for j in range(size):
            ans[i, j] = 1.0 / (i + j + 1)
    return ans

def gen_right_column(size):
    return np.array(np.random.random(size), dtype=np.float64)

In [11]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [12]:
def zeidelMethod(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            
    print("Iterations = ", curIterations)
    return xn

In [13]:
def zeidelMethodRelax(A, b):
    n = len(A)
    x = [.0 for i in range(n)]
    xn = [100 for i in range(n)]
    Relax = 1.6
    curIterations = 0
    while not equalVector(x, xn, EPS) and curIterations < ITERATIONS:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            xn[i] = Relax * xn[i] + (1 - Relax) * x[i]
            
    print("Iterations = ", curIterations)
    return xn

In [49]:
def gaussingMethod(A, b) :
    n = len(A)
    m = len(A[0])
    where = [-1 for x in range(m)]
    col = 0
    row = 0
    while col<m and row<n :
        sel = row
        for i in range(row, n) :
            if (abs(A[i][col]) > abs(A[sel][col])) :
                sel = i
        if (abs(A[sel][col]) < EPS) :
            col += 1
            continue
        for i in range(col, m) :
            A[sel][i], A[row][i] = A[row][i], A[sel][i]
        b[sel], b[row] = b[row], b[sel]
        where[col] = row
        
        for i in range(n) :
            if i != row :
                c = A[i][col] / A[row][col]
                for j in range(col, m) :
                    A[i][j] -= A[row][j] * c
                b[i] -= b[row] * c  
        col += 1
        row += 1
        
    ans = [0 for x in range(m)]
    for i in range (m) :
        if where[i] != -1 :
            ans[i] = b[where[i]] / A[where[i]][i]
    for i in range (n) :
        sum = 0
        for j in range (m) :
            sum += ans[j] * A[i][j]
    return ans

In [50]:
def _test(method, matrix, column):
    return method(matrix, column)

def _check_eps(a, b, eps):
    return abs(a - b) < eps
EPSILON = 1e-2
def _check_ans(matrix, column, res, size):
    eps = EPSILON
    c = np.matmul(matrix, res)
    for i in range(size):
        if not _check_eps(c[i], column[i], eps):
            return False
    return True

def _testAll(matrix_gen, size, methods):
    matrix, column = matrix_gen(size), gen_right_column(size)
    print(f'Тест для матрицы {matrix} и столбца {column}')
    for method in methods:
        print(f'Результат выполнения метода {method.__name__}:')
        res = _test(method, matrix, column)
        check = _check_ans(matrix, column, res, size)
        if check:
            print(f'{res}. Погрешность не больше {EPSILON}\n')
        else:
            print(f'разошлось: matrix * res = {np.matmul(matrix, res)}\n ожидаемый столбец = {column}\n')

def _get_methods():
    return [zeidelMethod, zeidelMethodRelax, gaussingMethod]

def testDiagMatrix(size):
    _testAll(diag_matrix, size, _get_methods())

def testHilbertMatrix(size):
    _testAll(hilbert_matrix, size, _get_methods())
    
def testRandMatrix(size):
    _testAll(rand_matrix, size, _get_methods())

In [51]:
testDiagMatrix(10)

Тест для матрицы [[44.62712166  4.51377357  5.03843335  0.95362166  9.26368194  2.38038724
   9.0799795   0.28210302  5.90729108  0.19138213]
 [ 6.11249621 54.99742349  9.53815859  2.71470049  1.75936089  3.57477987
   7.20071099  9.39601656  0.93532389  4.50943966]
 [ 6.44115265  5.80957837 51.63354338  4.64576859  4.01550661  6.36256034
   5.2458284   2.07922999  3.42372542  6.29226668]
 [ 6.39012309  1.2797045   4.13008254 41.03777032  2.19641135  6.65222493
   8.31265652  2.14732249  2.48554979  2.62024177]
 [ 0.64590913  2.29380842  9.98078175  3.3503148  42.03905544  8.21382834
   1.37924934  6.74893199  5.15283995  0.37902263]
 [ 5.9439523   9.02968451  2.88890686  8.28719114  7.45275378 54.15322242
   7.22685027  1.64596968  3.22570116  5.68025268]
 [ 0.11832737  0.65023377  2.1409168   7.64491881  8.77854649  1.81733492
  36.27298729  7.85956972  1.47372038  4.20210983]
 [ 0.92116463  1.90232578  6.15174905  6.24608042  5.02266052  9.12363213
   6.62698861 57.37377324  8.19941

In [52]:
testRandMatrix(4)

Тест для матрицы [[0.63537908 0.28309394 0.05033042 0.07792131]
 [0.28309394 0.50839381 0.84415231 0.86062795]
 [0.05033042 0.84415231 0.13992388 0.73829609]
 [0.07792131 0.86062795 0.73829609 0.48698314]] и столбца [0.71869555 0.61554369 0.99315219 0.30792498]
Результат выполнения метода zeidelMethod:


/home/anka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/anka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in double_scalars
  if sys.path[0] == '':
/home/anka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


Iterations =  200000
разошлось: matrix * res = [nan nan nan nan]
 ожидаемый столбец = [0.71869555 0.61554369 0.99315219 0.30792498]

Результат выполнения метода zeidelMethodRelax:


/home/anka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/anka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


Iterations =  200000
разошлось: matrix * res = [nan nan nan nan]
 ожидаемый столбец = [0.71869555 0.61554369 0.99315219 0.30792498]

Результат выполнения метода gaussingMethod:
[0.890059681277676, 0.4195833437199731, -0.7963555382215995, 0.9557033125101474]. Погрешность не больше 0.01



In [53]:
testHilbertMatrix(5)

Тест для матрицы [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]] и столбца [0.30394191 0.45578112 0.13113997 0.55401545 0.07549049]
Результат выполнения метода zeidelMethod:
Iterations =  200000
[-719.3414472360529, 13555.839522936973, -58744.626979064655, 89006.39162451038, -43641.64972660375]. Погрешность не больше 0.01

Результат выполнения метода zeidelMethodRelax:
Iterations =  109027
[-719.5014397932699, 13558.776481278408, -58757.13946682608, 89025.12479250703, -43650.75450725804]. Погрешность не больше 0.01

Результат выполнения метода gaussingMethod:
[-719.5014439508767, 13558.776553541687, -58757.139763797815, 89025.12522576516, -43650.75471375027]. Погрешность не больше 0.01

